In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods, linkage_vis
from bending_validation import suppress_stdout as so
from elastic_rods import EnergyType, InterleavingType

### Writing the topology

In [ ]:
vertices = []
edges = []
for i in range(10):
    for j in range(4):
        theta = np.pi / 2 * j
        vertices.append([np.cos(theta), np.sin(theta), i])
        vertices.append([np.cos(theta + np.pi / 4), np.sin(theta + np.pi / 4), i + 0.5])
        edges.append([i * 8 + j*2, i * 8 + j*2 + 1])
        edges.append([i * 8 + j*2, i * 8 + (j*2 - 1)%8])
        if i != 9:
            edges.append([i * 8 + j*2 + 1, i * 8 + j*2 + 8])
            edges.append([i * 8 + (j*2 - 1)%8, i * 8 + j*2 + 8])

In [ ]:
with open('cylinder.obj', 'w') as f:
    for vx in vertices:
        f.write('v {} {} {}\n'.format(vx[0], vx[1], vx[2]))
    for e in edges:
        f.write('l {} {}\n'.format(e[0] + 1, e[1] + 1))

### Initialize the linkage and compute equilibrium

In [ ]:
l = elastic_rods.RodLinkage('cylinder.obj', 8, rod_interleaving_type = InterleavingType.xshell)
driver=l.centralJoint()
l.setMaterial(elastic_rods.RodMaterial('rectangle', 20000, 0.3, [0.1,0.01]))

# Removing the rest curvatures from initialization so the beams are straight at rest state.
for i in range(l.numSegments()):
    r = l.segment(i).rod
    r.setRestKappas(np.zeros_like(r.restKappas()))

jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with so(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)
view = linkage_vis.LinkageViewer(l, width=1024, height=640)
view.setCameraParams(((2.3969, -1.1451, -0.5262), (0.0765, 0.0848, -0.9935), (0.1111, 0.1865, 0.5316)))
view.show()

In [ ]:
view.averagedMaterialFrames = True
view.averagedCrossSections = True

### Deploy the linkage

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.gradTol = 1e-4
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
with so(): open_linkage(l, driver, np.deg2rad(93) - l.averageJointAngle, 40, view, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, useTargetAngleConstraint=True);

### Render a high res image

In [ ]:
orender = view.offscreenRenderer(width=4096, height=2560)
orender.render()
orender.image()

In [ ]:
orender.save('cylinder.png')